In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

# Khởi tạo SparkSession
spark = SparkSession.builder.appName("05-dataframe-join").master("local[*]").getOrCreate()

# Đọc dữ liệu từ các tệp JSON
df1 = spark.read \
    .format("json") \
    .option("inferSchema", "true") \
    .load("data/d1/*")

df2 = spark.read \
    .format("json") \
    .option("inferSchema", "true") \
    .load("data/d2/*")

# In ra schema của cả hai DataFrame
df1.printSchema()
df2.printSchema()


root
 |-- DEST: string (nullable = true)
 |-- DEST_CITY_NAME: string (nullable = true)
 |-- FL_DATE: string (nullable = true)
 |-- OP_CARRIER: string (nullable = true)
 |-- OP_CARRIER_FL_NUM: long (nullable = true)
 |-- ORIGIN: string (nullable = true)
 |-- ORIGIN_CITY_NAME: string (nullable = true)
 |-- id: long (nullable = true)

root
 |-- ARR_TIME: long (nullable = true)
 |-- CANCELLED: long (nullable = true)
 |-- CRS_ARR_TIME: long (nullable = true)
 |-- CRS_DEP_TIME: long (nullable = true)
 |-- DEP_TIME: long (nullable = true)
 |-- DISTANCE: long (nullable = true)
 |-- TAXI_IN: long (nullable = true)
 |-- WHEELS_ON: long (nullable = true)
 |-- id: long (nullable = true)



In [3]:
df1.show(1)

+----+--------------+--------+----------+-----------------+------+----------------+---+
|DEST|DEST_CITY_NAME| FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|ORIGIN_CITY_NAME| id|
+----+--------------+--------+----------+-----------------+------+----------------+---+
| ATL|   Atlanta, GA|1/1/2000|        DL|             1451|   BOS|      Boston, MA|  0|
+----+--------------+--------+----------+-----------------+------+----------------+---+
only showing top 1 row



In [16]:
df1.join(df2, on = "id", how = "left").filter((col("CANCELLED") == 1) & (year(to_date("FL_DATE", "d/M/yyyy")).isNotNull())) \
    .groupBy("DEST",year(to_date("FL_DATE", "d/M/yyyy")).alias("FL_YEAR")).count() \
    .withColumnRenamed("count","NUM_CANCELLED_FLIGHT") \
    .orderBy(col("DEST"),col("FL_YEAR")) \
    .show()

+----+-------+--------------------+
|DEST|FL_YEAR|NUM_CANCELLED_FLIGHT|
+----+-------+--------------------+
| ABE|   2000|                   5|
| ABQ|   2000|                  15|
| AGS|   2000|                   1|
| ALB|   2000|                  12|
| AMA|   2000|                   5|
| ANC|   2000|                  36|
| ATL|   2000|                 239|
| AUS|   2000|                  30|
| AVL|   2000|                   3|
| AVP|   2000|                   2|
| AZO|   2000|                   1|
| BDL|   2000|                  18|
| BET|   2000|                   2|
| BHM|   2000|                   6|
| BIL|   2000|                   3|
| BIS|   2000|                   2|
| BNA|   2000|                  15|
| BOI|   2000|                   9|
| BOS|   2000|                 156|
| BRW|   2000|                   3|
+----+-------+--------------------+
only showing top 20 rows

